In [ ]:

from config import DB_NAME
import sqlite3, os
from scrapers.db import initialise_db
#initialise database if not exists
if not os.path.exists(DB_NAME):
    initialise_db(DB_NAME) 



In [1]:
from scrapers.culture_amp_scraper import CultureAmpScraper


scraper = CultureAmpScraper("CultureAmp", "https://www.cultureamp.com/company/careers#open-roles")
jobs = scraper.fetch_jobs()

In [5]:
# Append job details to jobs list dataframe
import pandas as pd
jobs_df = pd.DataFrame(columns=["Title", "Location", "Link"])
for job in jobs:
    title, location = job.select('p')
    link = job.get('href')
    jobs_df = pd.concat([jobs_df, pd.DataFrame({"Title": [title.text.strip()], "Location": [location.text.strip()], "Link": [link]})], ignore_index=True)

#save into db
scraper.save_jobs(jobs_df, DB_NAME)